In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

features_train = train_df.drop(['SalePrice'], axis=1)
target_train = train_df['SalePrice']

features_test = test_df

num_features = features_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = features_train.select_dtypes(include=['object']).columns.tolist()

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocess_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features)
    ])

features_train_prepared = preprocess_pipeline.fit_transform(features_train)
features_test_prepared = preprocess_pipeline.transform(features_test)

features_train_prepared = features_train_prepared.toarray()
features_test_prepared = features_test_prepared.toarray()

neural_model = Sequential([
    Dense(64, activation='relu', input_shape=(features_train_prepared.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)
])

neural_model.compile(optimizer=Adam(), loss='mse')

neural_model.fit(features_train_prepared, target_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

predictions_test = neural_model.predict(features_test_prepared)

Epoch 1/100
37/37 [==============================] - 1s 11ms/step - loss: 38817411072.0000 - val_loss: 39911817216.0000
Epoch 2/100
37/37 [==============================] - 0s 9ms/step - loss: 38796439552.0000 - val_loss: 39866896384.0000
Epoch 3/100
37/37 [==============================] - 0s 9ms/step - loss: 38707736576.0000 - val_loss: 39720538112.0000
Epoch 4/100
37/37 [==============================] - 0s 8ms/step - loss: 38479052800.0000 - val_loss: 39397605376.0000
Epoch 5/100
37/37 [==============================] - 0s 9ms/step - loss: 38029271040.0000 - val_loss: 38815649792.0000
Epoch 6/100
37/37 [==============================] - 0s 11ms/step - loss: 37270335488.0000 - val_loss: 37900754944.0000
Epoch 7/100
37/37 [==============================] - 0s 11ms/step - loss: 36145836032.0000 - val_loss: 36588507136.0000
Epoch 8/100
37/37 [==============================] - 0s 7ms/step - loss: 34598879232.0000 - val_loss: 34871353344.0000
Epoch 9/100
37/37 [==========================

In [10]:
predictions_train = neural_model.predict(features_train_prepared)
print("Mean Absolute Error:", mean_absolute_error(target_train, predictions_train))
print("Mean Squared Error:", mean_squared_error(target_train, predictions_train))
print("R^2 Score:", r2_score(target_train, predictions_train))

predictions_submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': predictions_test.flatten()
})

predictions_submission.to_csv('predictions.csv', index=False)

46/46 [==============================] - 0s 2ms/step
Mean Absolute Error: 18052.47382411173
Mean Squared Error: 1034864033.2983509
R^2 Score: 0.835912680551502
